# Test API "Flood warnings"

In [1]:
import requests
import json
import pandas as pd
import folium
import geopandas as gpd

In [2]:
url = 'http://environment.data.gov.uk/flood-monitoring/id/floods'

### à remplir : latitude, longitude et radius (en km) du point de départ

In [3]:
params = {
    #'lat': 51.507581,
    #'long': -0.127597,
    #'distance': 5
}

### requête

In [4]:
try:
    api_request = requests.get(url, params=params)
    data = json.loads(api_request.content)
except:
    data = None

### passage à un DataFrame

In [5]:
df = pd.DataFrame(data['items'])

df.head(10)

,@id,description,eaAreaName,eaRegionName,floodArea,floodAreaID,isTidal,message,severity,severityLevel,timeMessageChanged,timeRaised,timeSeverityChanged
0,http://environment.data.gov.uk/flood-monitorin...,Upper Bristol Avon area,Wessex,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,112WAFTUBA,False,River levels are now falling on the Bristol Av...,Flood alert,3,2024-01-01T10:59:00,2024-01-01T10:59:53,2023-12-28T08:54:00
1,http://environment.data.gov.uk/flood-monitorin...,River Blackwater and The Cove Brook,Thames,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,061WAF24BlkWater,False,Property flooding is not currently expected. R...,Flood alert,3,2024-01-07T13:39:00,2024-01-07T13:39:19,2024-01-02T12:40:00
2,http://environment.data.gov.uk/flood-monitorin...,River Lavant,Solent and South Downs,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,065WAF413,False,"The River Lavant is high, but stable following...",Warning no longer in force,4,2024-01-07T11:25:00,2024-01-07T11:25:25,2024-01-07T11:25:00
3,http://environment.data.gov.uk/flood-monitorin...,"Upper River Derwent, Stonethwaite Beck and Der...",Cumbria and Lancashire,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,011WAFDW,False,,Warning no longer in force,4,2024-01-07T09:37:00,2024-01-07T09:37:20,2024-01-07T09:37:00
4,http://environment.data.gov.uk/flood-monitorin...,River Churn and its tributaries,Thames,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,061WAF02Churn,False,Property flooding is not currently expected. ...,Flood alert,3,2024-01-07T10:49:00,2024-01-07T10:50:06,2023-11-02T07:13:00
5,http://environment.data.gov.uk/flood-monitorin...,River Cherwell from Lower Heyford down to and ...,Thames,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,061WAF14LChrwell,False,Property flooding is not expected. The flood ...,Flood alert,3,2024-01-07T11:13:00,2024-01-07T11:13:31,2023-12-28T17:09:00
6,http://environment.data.gov.uk/flood-monitorin...,Upper River Ouse,Yorkshire,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,122WAF946,False,River levels at the York Viking Recorder on th...,Flood alert,3,2024-01-08T04:07:00,2024-01-08T04:07:18,2023-12-27T20:58:00
7,http://environment.data.gov.uk/flood-monitorin...,Severn Vyrnwy confluence,West Midlands,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,031WAF114,False,River levels remain high but are falling stea...,Flood alert,3,2024-01-07T17:04:00,2024-01-07T17:04:42,2023-12-07T16:46:00
8,http://environment.data.gov.uk/flood-monitorin...,Upper River Derwent,Yorkshire,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,122WAF949,False,River levels are falling slowly but remain hig...,Flood alert,3,2024-01-08T04:13:00,2024-01-08T04:13:34,2023-12-04T19:59:00
9,http://environment.data.gov.uk/flood-monitorin...,River Evenlode from Moreton in Marsh to Cassin...,Thames,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,061WAF12Evenlode,False,Property flooding is not currently expected. ...,Flood alert,3,2024-01-07T11:13:00,2024-01-07T11:13:24,2023-12-30T18:20:00


### nettoyage des données

In [6]:
# on enlève les colonnes inutiles
columns_to_drop = ['eaRegionName', 'isTidal', 'timeSeverityChanged', 'timeMessageChanged']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

In [7]:
# on ne garde que le polygone
df['floodArea'] = df['floodArea'].apply(lambda x: x['polygon'] if 'polygon' in x else None)

### alertes sur la dernière heure

In [18]:
# affichage des alertes sur la dernière heure
#last_hour = df[df['severityLevel'].isin([3,4])].sort_values(by=['timeRaised']).tail(30)
last_hour = df.sort_values(by=['timeRaised']).tail(15)

last_hour

,@id,description,eaAreaName,floodArea,floodAreaID,message,severity,severityLevel,timeRaised
120,http://environment.data.gov.uk/flood-monitorin...,River Trent at Carlton on Trent including Beck...,East Midlands,http://environment.data.gov.uk/flood-monitorin...,034FWBTRCARLTON,River levels remain high at the Torksey river...,Flood warning,2,2024-01-08T07:40:06
121,http://environment.data.gov.uk/flood-monitorin...,River Trent at Collingham,East Midlands,http://environment.data.gov.uk/flood-monitorin...,034FWBTRCOLLHAM,River levels remain high but are beginning t...,Flood warning,2,2024-01-08T07:40:10
165,http://environment.data.gov.uk/flood-monitorin...,River Trent at Dunham-on-Trent,East Midlands,http://environment.data.gov.uk/flood-monitorin...,034FWBTRDUNHAM,River levels remain high but are beginning to...,Flood warning,2,2024-01-08T07:40:18
112,http://environment.data.gov.uk/flood-monitorin...,River Trent at Girton,East Midlands,http://environment.data.gov.uk/flood-monitorin...,034FWBTRGIRTON,River levels remain high but are beginning to ...,Flood warning,2,2024-01-08T07:40:23
368,http://environment.data.gov.uk/flood-monitorin...,River Trent at Sutton-on-Trent including Carlt...,East Midlands,http://environment.data.gov.uk/flood-monitorin...,034FWBTRSUTTON,River levels remain high but are beginning to...,Flood warning,2,2024-01-08T07:40:27
308,http://environment.data.gov.uk/flood-monitorin...,River Trent at High Marnham and Low Marnham,East Midlands,http://environment.data.gov.uk/flood-monitorin...,034FWBTRMARNHAM,River levels remain high but are beginning to ...,Flood warning,2,2024-01-08T07:40:36
359,http://environment.data.gov.uk/flood-monitorin...,River Trent at Besthorpe,East Midlands,http://environment.data.gov.uk/flood-monitorin...,034FWBTRBESTHRP,River levels remain high but are beginning to ...,Flood warning,2,2024-01-08T07:40:37
23,http://environment.data.gov.uk/flood-monitorin...,Upper River Hull catchment,Yorkshire,http://environment.data.gov.uk/flood-monitorin...,122WAF972,River levels remain high but steady on Beverle...,Flood alert,3,2024-01-08T07:50:55
51,http://environment.data.gov.uk/flood-monitorin...,River Hull Central catchment,Yorkshire,http://environment.data.gov.uk/flood-monitorin...,122WAF973,River levels are high on Beverley and Barmston...,Flood alert,3,2024-01-08T07:50:59
103,http://environment.data.gov.uk/flood-monitorin...,Hundred Foot Washes in Cambridgeshire and Norfolk,East Anglia,http://environment.data.gov.uk/flood-monitorin...,052WAFHFW,River levels are rising in response to recent ...,Flood alert,3,2024-01-08T07:53:30


### map pour les dernières alertes

In [22]:
# carte avec vue centrée sur le Royaume-Uni
m = folium.Map(location=[53.5, -1.5], zoom_start=6)

# on itère sur chaque ligne du dataframe avec iterrows()
for index, row in last_hour.iterrows():
    # on récupère le chemin du fichier geojson pour le polygone
    geojson_file = row['floodArea']
    gdf = gpd.read_file(geojson_file)

    # on récupère le niveau de gravité pour la ligne actuelle
    severity_level = row['severityLevel']
    
    # on définit la couleur en fonction du niveau de gravité
    if severity_level == 1:
        color = 'green'
    elif severity_level == 2:
        color = 'yellow'
    elif severity_level == 3:
        color = 'orange'
    elif severity_level == 4:
        color = 'red'
    else:
        color = 'blue' # couleur par défaut mais normalement pas besoin 
    
    # on ajoute le polygone à la carte avec la couleur correspondante
    geo_j = folium.GeoJson(
        data=gdf, 
        style_function=lambda x, color=color: {'fillColor': color, 'color': color} 
    )
    geo_j.add_to(m)

# affichage de la carte
display(m)